In [4]:
import numpy as np
import os
import scipy.io as sio
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance
from matplotlib import pyplot as plt

In [6]:
# First, run the code in extract_precision_matrices.ipynb to obtain precision matrices: FCprec_concat_allsub.mat
# Then, run unpack_precision_mats to split FCprec_concat_allsub.mat into single subject .mat files

cwd='/Users/emilyolafson/GIT/stroke-graph-matching/data/'
unpack_precision_mats(cwd)

# After unpack_precision_mats is run, format precision data:
# - Set diagonals of precision FC to 0.
# - create a .mat file that contains precision FCs in a cell array. Subjects in rows & longitudinal time points in columns.

# MATLAB code for formatting this dataset: format_precision.m

# Then, 

In [2]:
def unpack_precision_mats(cwd):
    """Turn concatenated precision FC matrix into multiple single subject/single session matrices.
    Save each single suject to /precision/ folder."""
    data_dir = str(cwd) 
    subj_all = np.genfromtxt(data_dir + 'controls/precision/subjects_ctl.txt', dtype = 'str') #list of subjects in same order as precision matrices are saved.    
    fc_dir = data_dir + 'controls/precision/'
    output_prec = sio.loadmat(fc_dir + 'FCprec_concat_allsub.mat')
    prec = output_prec['C'] 
    c = 0
    for sub in subj_all:
        mdic = {"C": prec[c]}
        sio.savemat(fc_dir + sub, mdic)
        c=c+1
        
def remap_cost_regularized(interval, fc, alpha, beta):
    """Cost function: the cost of remapping each node in matrix A to every other node in matrix B. 
    Output is fed into graph matching algorithm. Regularization: (costmatrix - alpha*I) 
    where alpha is a regularization parameter controlling the degree to which a remapping
    to itself is preferred over remapping to any other node."""
    cwd = os.getcwd()
    data_dir = '/Users/emilyolafson/GIT/stroke-graph-matching/data'
    distances=sio.loadmat(data_dir + '/pairwise_eucl_dist_shen268.mat')

    distances=distances['eucl_dist']
    costmat_all=list()

    nROIs=fc[0][0].shape[0]
    
    #calculate cost between all ROIs
    for i in range(0,fc.shape[0]): #subject 

            
        costmat=np.zeros((nROIs,nROIs))

        for x in range(0,nROIs): #x = time point 1.
            a=fc[i,interval[0]][x]

            for y in range(0,nROIs): #y = time point 2.
                b=fc[i,interval[1]][y]
                
                costmat[x,y]=distance.euclidean(a,b)
                
        costmat=costmat+beta*distances
        costmat_all.append(costmat)

    return [costmat_all, nROIs]

def graph_matching_regularized(costmat_all, interval, nROIs, dx, alpha, alphacounter, beta, betacounter):
    """Runs graph matching w/ the Hungarian algorithm and saves outputs:
    
        cols_SXSY.txt - each row is a different subject. Values in each column represent the node
        in the latter time point that the node in the prior time point was mapped to.
        
        roichanges_SXSY.txt - each row is a different subject. Values in each column represent 
        whether the node was remapped to a DIFFERENT node than itself in the subsequent time point 
        (elements are 0 or 1, 1 if the node was remapped to a different node and 0 if the node was mapped to itself.
        Used to calculate remap frequency for each region, when averaged vertically (across subjects)."""
    
    results_dir = '/Users/emilyolafson/GIT/stroke-graph-matching/project/results/controls'
    print(alphacounter)
    nROIs=fc[0][0].shape[0]
    nsubs=len(costmat_all) #test
    
    rows=np.zeros((nsubs,nROIs))
    cols=np.zeros((nsubs,nROIs))
    roichanges=np.zeros((nsubs,nROIs))
    truecols=range(0,nROIs)

    for i in range(0, nsubs):
        cost=costmat_all[i]
        rowind, colind=linear_sum_assignment(cost*10000) #graph matching part.
        rows[i]=rowind
        cols[i]=colind

    np.savetxt(results_dir+'/precision/cols_'+ 'S'+str(interval[0]+1)+'S'+ str(interval[1]+1)+'_alpha' + str(alphacounter)+'_beta' + str(betacounter) + '.txt', cols)

    for j in range(0,nsubs):
        for i in range(0,nROIs):
            if cols[j][i]!=truecols[i]:
                roichanges[j][i]=1 #indices that are switched

    allchanges=np.mean(roichanges,0)
    np.savetxt(results_dir +'/precision/roichanges_' + 'S'+str(interval[0]+1)+'S'+ str(interval[1]+1) +'_alpha' + str(alphacounter)+'_beta' + str(betacounter) +'.txt', allchanges)
    return [allchanges]

def run_graph_matching_reg(interval, fc, dx, alpha,alphacounter, beta, betacounter):
    """Calculate the cost of remapping across all subjects, and perform graph matching.
    Saves outputs to /results/jupyter/.../stroke and /controls."""
    [costmat_all, nROIs]=remap_cost_regularized(interval, fc, alpha, beta)
    nswaps=graph_matching_regularized(costmat_all, interval, nROIs, dx, alpha,alphacounter, beta, betacounter)
    return nswaps, costmat_all


In [6]:
cwd = os.getcwd()
data_dir = '/Users/emilyolafson/GIT/stroke-graph-matching/data'
fc_dir =data_dir + '/controls/precision'
fc=sio.loadmat(fc_dir + '/C_precision.mat')
fc=fc['C_precision']

dx='stroke'
intervals=[[0, 1], [1, 2], [2, 3], [3, 4]] #time point comparisons. 0=1, 1=2, etc.
alpha=0
session_swaps=list()
for i in range(0, 4):
    allswaps=list()
    betas=[0, 0.0001, 0.0002, 0.0003, 0.0004]
    betacounter = -1
    alphacounter = 0
    for beta in betas:
        betacounter = betacounter +1
        print('Testing beta: ' + str(beta))
        [nswaps, costmat_all]=run_graph_matching_reg(intervals[i], fc, dx, alpha, alphacounter, beta, betacounter)
        print('Mean num. swaps: ' + str(np.mean(nswaps)))
        mdic = {"cost": costmat_all}
        results_dir = '/Users/emilyolafson/GIT/stroke-graph-matching/project/results/controls/'
        sio.savemat(results_dir + 'costmatrix_10000.mat', mdic)
        allswaps.append(nswaps)

    
    session_swaps.append(allswaps)

Testing beta: 0
0
Mean num. swaps: 0.4001865671641791
Testing beta: 0.0001
0
Mean num. swaps: 0.08893034825870647
Testing beta: 0.0002
0
Mean num. swaps: 0.05332711442786069
Testing beta: 0.0003
0
Mean num. swaps: 0.03373756218905473
Testing beta: 0.0004
0
Mean num. swaps: 0.026119402985074626
Testing beta: 0
0
Mean num. swaps: 0.406094527363184
Testing beta: 0.0001
0
Mean num. swaps: 0.10898631840796019
Testing beta: 0.0002
0
Mean num. swaps: 0.06094527363184079
Testing beta: 0.0003
0
Mean num. swaps: 0.04197761194029851
Testing beta: 0.0004
0
Mean num. swaps: 0.0343594527363184
Testing beta: 0
0
Mean num. swaps: 0.384794776119403
Testing beta: 0.0001
0
Mean num. swaps: 0.10074626865671642
Testing beta: 0.0002
0
Mean num. swaps: 0.05674751243781094
Testing beta: 0.0003
0
Mean num. swaps: 0.042754975124378106
Testing beta: 0.0004
0
Mean num. swaps: 0.035292288557213926
Testing beta: 0
0
Mean num. swaps: 0.3571206467661691
Testing beta: 0.0001
0
Mean num. swaps: 0.09343905472636815
Test

In [17]:
[cwd='/Users/emilyolafson/GIT/stroke-graph-matching'
data_dir = str(cwd) + '/results/'
counter = 0
for i in session_swaps:
    swaps=i[:][:][:]
    meanswaps=np.mean(swaps,axis=2)
    mdic = {"meanswaps": meanswaps}
    sio.savemat(data_dir + 'stroke_meanswaps' + str(counter) + '.mat', mdic)
    counter = counter +1
    